In [2]:
import pandas as pd

In [318]:
dt_links=pd.read_csv("links.csv",encoding="utf-8")
dt_movies=pd.read_csv("movies.csv",encoding="utf-8")
dt_tags=pd.read_csv("tags.csv",encoding="utf-8")
dt_ratings=pd.read_csv("ratings.csv",encoding="utf-8")
parsed_films=pd.read_csv("films_parsed.csv",encoding="utf-8")
user_films=pd.read_csv("users_films.csv",encoding="utf-8")

In [247]:
user_films.pop('userId')

0        1.0
1        2.0
2        3.0
3        4.0
4        5.0
       ...  
605    606.0
606    607.0
607    608.0
608    609.0
609    610.0
Name: userId, Length: 610, dtype: float64

In [ ]:
users_films=user_films.copy

![vase](wr.png)

v количество отзывов 
m минимальное количесчтво отзывов для чарта
R средний рейтинг фильм
C средний рейтинг по всем фильмам

In [30]:
def getAverageRatings(users_films, if_user):
    """Funtion calculating average rating by user or for film

    Args:
        users_films (DataFrame): Table with users and films they rated/viewed.
        if_user (bool): this telling us calculate for user/film.

    Returns:
        dict: Avarege rating for each movie or user.

    """

    ax = 1 if if_user else 0
    #axis = 1 means rows and axis = 0 means columns 
    sumOfRatings = users_films.sum(axis = ax).to_numpy() #this will give an array of sum of all the ratings of user if axis = 1 else 
    #sum of all the ratings of movies if axis = 0
    noOfRatings = (users_films!=0).sum(axis = ax).to_numpy()  #this will give a boolean True or False array, and True means 1 and False 
    #means 0, and further we are summing it to get the count of all the non-zero cells means length of non-zero cells
    rows, cols = users_films.shape
    averageRatings = {i: sumOfRatings[i]/noOfRatings[i] for i in range(rows if if_user else cols) if noOfRatings[i]!=0}
    return averageRatings

In [62]:
avg_raring_per_movie = list(getAverageRatings(user_films, False).values())

In [68]:
def Average(lst):
    return sum(lst) / len(lst)

In [69]:
C=Average(avg_raring_per_movie)

In [78]:
dt_ratings_2= dt_ratings.groupby(['movieId']).count()

In [80]:
dt_ratings_2=dt_ratings_2.loc[dt_ratings_2['userId'] > 1]

In [84]:
m=dt_ratings_2['userId'].quantile(0.9)

In [85]:
q_movies = dt_ratings_2.copy().loc[dt_ratings_2['userId'] >= m]

In [88]:
q_movies

,userId,rating,timestamp
movieId,,,
1,215,215,215
2,110,110,110
3,52,52,52
5,49,49,49
6,102,102,102
...,...,...,...
122882,47,47,47
122886,41,41,41
122904,54,54,54


In [89]:
avg_raring_per_movie = getAverageRatings(user_films, False)

In [105]:
movies_merged=pd.merge(dt_movies, dt_ratings.groupby(['movieId']).count(), on='movieId', how='inner')

In [106]:
movies_merged=movies_merged.loc[movies_merged['userId'] > 1]

In [107]:
movies_merged['avg']=list(getAverageRatings(user_films, False).values())

In [104]:
len(list(getAverageRatings(user_films, False).values()))

6278

In [110]:
m=movies_merged['userId'].quantile(0.9)

In [111]:
q_movies = movies_merged.copy().loc[movies_merged['userId'] >= m]

In [113]:
def weighted_rating(x, m=m, C=C):
    v = x['userId']
    R = x['avg']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [114]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)


In [115]:
q_movies = q_movies.sort_values('score', ascending=False)


In [120]:
q_movies

,movieId,title,genres,userId,rating,timestamp,avg,score
2037,2716,Ghostbusters (a.k.a. Ghost Busters) (1984),Action|Comedy|Sci-Fi,120,120,120,4.666667,4.319590
4791,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,185,185,185,4.375000,4.180042
126,153,Batman Forever (1995),Action|Adventure|Comedy|Crime,137,137,137,4.350000,4.107822
1266,1682,"Truman Show, The (1998)",Comedy|Drama|Sci-Fi,125,125,125,4.323529,4.070155
6298,48516,"Departed, The (2006)",Crime|Drama|Thriller,107,107,107,4.357143,4.063597
...,...,...,...,...,...,...,...,...
7439,81591,Black Swan (2010),Drama|Thriller,42,42,42,1.125000,2.175420
6693,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,149,149,149,1.750000,2.073463
1729,2324,Life Is Beautiful (La Vita è bella) (1997),Comedy|Drama|Romance|War,88,88,88,1.500000,2.055738
1733,2329,American History X (1998),Crime|Drama,129,129,129,1.500000,1.920914


In [133]:
q_movies.loc[q_movies['movieId']==7153].index[0]

4791

In [166]:
def find_no_viewed(x,k):
    if user_films[str(x)][k]==0:
        return x
    else:
        pass
    

In [183]:
def recommend_for_user_k(k):
    recomend=[]
    for i in q_movies['movieId']:
            if user_films[str(i)][k-1]==0:
                recomend.append(q_movies.loc[q_movies['movieId']==i].index[0])
            if len(recomend) == 10:
                return recomend
    

In [466]:
recommend_for_user_k(1)

[4791, 126, 1266, 6298, 3609, 4402, 277, 3539, 504, 276]

In [203]:
getAverageRatings(user_films, True)[0]

4.367965367965368

In [506]:
n=0
k=50
deleted=[]
for i in range (n,k):
    dict_={}
    p=10
    avg = getAverageRatings(user_films, True)[i]
    for j in movies_merged['movieId'].sample(frac = 1):
        if user_films[str(j)][i]>avg:
            dict_[(movies_merged[movies_merged['movieId']==j].index[0])]=user_films[str(j)][i]
            p=p-1
            user_films[str(j)][i]=0
        if p==0 :
            break
    deleted.append(dict_)

In [507]:
avg_prec=0.0
for user in range(n+1,k+1):
    count=0.0
    recs=recommend_for_user_k(user)
    for rec in recs:
        if users_films[str(movies_merged['movieId'][rec])][user-1]>0:
            count=count+1
    count=count/10
    print(count)
    avg_prec=avg_prec+count
print('avg precision:',avg_prec/10)
            

0.2
0.2
0.0
0.1
0.2
0.1
0.4
0.2
0.0
0.0
0.1
0.0
0.0
0.0
0.1
0.0
0.1
0.0
0.1
0.0
0.0
0.1
0.0
0.0
0.2
0.1
0.0
0.2
0.1
0.1
0.0
0.1
0.1
0.0
0.0
0.0
0.1
0.1
0.0
0.1
0.1
0.0
0.0
0.1
0.0
0.1
0.0
0.0
0.0
0.0
avg precision: 0.3400000000000002


0.1 avg precision на тестовой выборке